# Extracting Tweets

In [1]:
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import re
import datetime
import time
from textblob import TextBlob

In [2]:
def clean_tweet(txt):
    txt = re.sub(r'@[A-Za-z0-9_]+', "", txt)
    txt = re.sub(r'#', "", txt)
    txt = re.sub(r'\n', "", txt)
    txt = re.sub(r'RT :', "", txt)
    txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', "", txt)
    return txt

## MySQL Authorization

In [3]:
import mysql.connector
from mysql.connector import Error

from mysql_credentials import *

host = HOST
user = USER
password = PASSWORD
database = 'twitter'

## Create MySQL Functions  

### Connecting to MySQL Server

In [4]:
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host = host_name,
            user = user_name,
            passwd = user_password
        )
        print("MySQL Database connection successful!")
    except Error as err:
        print(f"Error: '{err}'")
    return connection

### Create Database Function

In [5]:
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully!")
    except Error as err:
        print(f"Error: '{err}'")

### Connecting to the Database 

In [6]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host = host_name,
            user = user_name,
            passwd = user_password,
            database = db_name
        )
        print("MySQL Database connection successful!")
    except Error as err:
        print(f"Error: '{err}'")
        
    return connection

### Create a Query Execution Function

In [7]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful!")
    except Error as err:
        print(f"Error: '{err}'")

### Select Function

In [8]:
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

### Insert Many Records Into SQL- Creating Records from Lists


In [9]:
def execute_list_query(connection, sql, val):
    cursor = connection.cursor()
    try:
        cursor.executemany(sql, val)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

## Create twitter Database and tweets Table

In [10]:
# Establish connection
connection = create_server_connection(host, user, password)

# Create twitter Database
#create_database(connection, "CREATE DATABASE twitter;")
create_database(connection, "CREATE DATABASE " + database + ";")

connection.close()

MySQL Database connection successful!
Database created successfully!


In [11]:
# Establish connection
connection = create_db_connection(host, user, password, database)

#Create tweets Table
sql = """
CREATE TABLE tweets(
id_str VARCHAR(255) NOT NULL,
username VARCHAR(255),
tweets VARCHAR(1024),
clean_tweets VARCHAR(1024),
created_at DATETIME,
location VARCHAR(255),
source VARCHAR(255),
likes INT,
retweet_count INT
);
"""
execute_query(connection, sql)

connection.close()

MySQL Database connection successful!
Query successful!


## Twitter: Connect to Twitter, Extract Tweets, and Save to MySQL

### Import Access Keys:


In [12]:
from TwitterNFTcredentials import * 

# Consumer:
consumer_key = CONSUMER_KEY
consumer_secret = CONSUMER_SECRET

# Access:
access_key = ACCESS_TOKEN
access_secret = ACCESS_SECRET

### Authorize with Tweepy's OAuthhandler:

In [13]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)

api = tweepy.API(auth)

### Extract Tweets:

In [ ]:
#hashtag_phrase = "#nfts OR #nft OR #nftart OR #nftartist OR #nftcollector OR #cryptoart OR #digitalart OR #nftcommunity OR #art OR #crypto OR #ethereum OR #cryptocurrency OR #cryptoartist OR #opensea OR #nftcollectors OR #nftdrop OR #nftcollectibles OR #artist OR #eth OR #openseanft OR #nftartists OR #artwork OR #artoftheday OR #raredigitalart OR #nftartgallery OR OR #bhfyp OR #animation OR #nftartwork OR #abstractart OR #rarible OR #cryptonews OR #nftcollection OR #cryptotrading OR #artgallery OR #btc OR #nftsstories OR #cryptocurrencies OR #digitalillustration OR #digitalartist OR #modernart OR #digitalcollectibles OR #music OR #superrare OR #pixelart OR #investing OR #illustration OR #artcollector OR #collectibles OR #binance OR #hicetnunc OR #cryptoworld OR #cryptoinvestor"
hashtag_phrase1 = "#nfts OR #nft OR #nftart OR #nftartist OR #nftcollector OR #cryptoart OR #digitalart OR #nftcommunity OR #art OR #crypto OR #ethereum OR #cryptocurrency OR #cryptoartist OR #opensea OR #nftcollectors OR #nftdrop OR #nftcollectibles OR #artist OR #eth OR #openseanft" 
hashtag_phrase2 = "#nftartists OR #artwork OR #artoftheday OR #raredigitalart OR #nftartgallery OR #bhfyp OR #animation OR #nftartwork OR #abstractart OR #rarible OR #cryptonews OR #nftcollection OR #cryptotrading OR #artgallery OR #btc OR #nftsstories OR #cryptocurrencies OR #digitalillustration" 
hashtag_phrase3 = "#digitalartist OR #modernart OR #digitalcollectibles OR #music OR #superrare OR #pixelart OR #investing OR #illustration OR #artcollector OR #collectibles OR #binance OR #hicetnunc OR #cryptoworld OR #cryptoinvestor"
hashtag_phrase = hashtag_phrase1

# Create a csv file for output
csv_file = open('NFTtweets.csv', 'a', encoding='utf-8') 
csv_writer = csv.writer(csv_file)
  

number_of_tweets = 500
id_str = []; username = []; tweets = []; clean_tweets = []; created_at = [];  
location = []; source = []; likes = []; retweet_count = []

ctr = 0
while ctr < 8:
    ctr = ctr + 1
    print ("Iteration number: = {}".format(ctr))

    id_str.clear(); username.clear(); tweets.clear(); clean_tweets.clear(); created_at.clear();  
    location.clear(); source.clear(); likes.clear(); retweet_count.clear()
    
    try:
        cursor = tweepy.Cursor(api.search_tweets, q=hashtag_phrase +' -filter:retweets',
                               lang="en", tweet_mode = "extended").items(number_of_tweets)

        for tweet in cursor:
            id_str.append(tweet.id_str)
            username.append(tweet.user.screen_name)
            tweets.append(tweet.full_text)
            clean_tweets.append(clean_tweet(tweet.full_text))
            created_at.append(tweet.created_at)
            location.append(tweet.user.location)
            source.append(tweet.source)
            likes.append(tweet.favorite_count)
            retweet_count.append(tweet.retweet_count)

            # Write the record
            csv_writer.writerow([tweet.id_str, tweet.user.screen_name, tweet.full_text, clean_tweet(tweet.full_text),
                                 tweet.created_at, tweet.user.location, tweet.source, tweet.favorite_count, 
                                 tweet.retweet_count])
         
        
        # Insert the Extracted Tweets into MySQL
        sql = '''
        INSERT INTO tweets(id_str, username, tweets, clean_tweets, created_at, location, source, likes, retweet_count)
        VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s)
        '''
        val = []

        for i in range(len(tweets)):
            val.append([id_str[i],
                        username[i], 
                        tweets[i], 
                        clean_tweets[i],
                        created_at[i], 
                        location[i], 
                        source[i], 
                        likes[i], 
                        retweet_count[i]
                       ])

        connection = create_db_connection(host, user, password, database)
        execute_list_query(connection, sql, val)
        connection.close()
        
        if ctr % 4 == 0:   # i.e. 4 iterations (500 tweets each). let's take an elective timeout!!!
            print(datetime.datetime.now().strftime('%H:%M:%S'))
            print("Elective timeout in order to avoid Twitter's 'status code = 429'")
            for i in range(3,0,-1):
                print('wait for {} minutes'.format(i*5))
                time.sleep(5*60)

    except tweepy.TooManyRequests as e:
        print(e.reason)
        for i in range(3,0,-1):
            print('wait for {} minutes'.format(i*5))
            time.sleep(5*60)
            
            
# Now we can save it to a csv file
csv_file.close() # close the csv file

Iteration number: = 1
MySQL Database connection successful!
Query successful
Iteration number: = 2
MySQL Database connection successful!
Query successful
Iteration number: = 3
MySQL Database connection successful!
Query successful
Iteration number: = 4
MySQL Database connection successful!
Query successful
13:55:55
Elective timeout in order to avoid Twitter's 'status code = 429'
wait for 15 minutes
wait for 10 minutes
wait for 5 minutes
Iteration number: = 5
MySQL Database connection successful!
Query successful
Iteration number: = 6
MySQL Database connection successful!
Query successful
Iteration number: = 7
MySQL Database connection successful!
Query successful
Iteration number: = 8
MySQL Database connection successful!
Query successful
14:13:23
Elective timeout in order to avoid Twitter's 'status code = 429'
wait for 15 minutes
